<a href="https://colab.research.google.com/github/asrivastava0601/HackerEarth_Attrition_Prediction/blob/master/Emp_attrition_rate_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import math

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import os

os.getcwd()

'/content'

In [ ]:
os.chdir("/content/gdrive/My Drive/HackerEarth_Predict_Emp_Attrition/Dataset")

In [ ]:
### Reading the training data ###
train_data = pd.read_csv("Train.csv")

In [ ]:
train_data.shape

(7000, 24)

In [ ]:
train_data.head()

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
0,EID_23371,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,33,1,1,7.0,type2,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841
1,EID_18000,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,36,0,3,6.0,type2,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670
2,EID_3891,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,51,0,2,8.0,type2,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851
3,EID_17492,F,26.0,3,Single,Lebanon,Human Resource Management,Behavioral,4.0,3,56,1,3,8.0,type2,1.0,3,-1.8176,-0.4537,NaN,3,7,3,0.0668
4,EID_22534,F,31.0,1,Married,Springfield,Logistics,Conceptual,5.0,4,62,1,3,2.0,type3,3.0,1,0.7516,-0.4537,2.0,2,8,2,0.1827


In [ ]:
### Check for NULL values ###
train_data.isnull().values.any()

True

In [ ]:
### Column/Feature wise total NULL values ###
train_data.isnull().sum()

Employee_ID                    0
Gender                         0
Age                          412
Education_Level                0
Relationship_Status            0
Hometown                       0
Unit                           0
Decision_skill_possess         0
Time_of_service              144
Time_since_promotion           0
growth_rate                    0
Travel_Rate                    0
Post_Level                     0
Pay_Scale                      9
Compensation_and_Benefits      0
Work_Life_balance             11
VAR1                           0
VAR2                         577
VAR3                           0
VAR4                         656
VAR5                           0
VAR6                           0
VAR7                           0
Attrition_rate                 0
dtype: int64

In [ ]:
desc = train_data.describe()

desc

,Age,Education_Level,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
count,6588.000000,7000.000000,6856.000000,7000.000000,7000.000000,7000.000000,7000.000000,6991.000000,6989.000000,7000.000000,6423.000000,7000.000000,6344.000000,7000.000000,7000.000000,7000.000000,7000.000000
mean,39.622799,3.187857,13.385064,2.367143,47.064286,0.817857,2.798000,6.006294,2.387895,3.098571,-0.008126,-0.013606,1.891078,2.834143,7.101286,3.257000,0.189376
std,13.606920,1.065102,10.364188,1.149395,15.761406,0.648205,1.163721,2.058435,1.122786,0.836377,0.989850,0.986933,0.529403,0.938945,1.164262,0.925319,0.185753
min,19.000000,1.000000,0.000000,0.000000,20.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-1.817600,-2.776200,1.000000,1.000000,5.000000,1.000000,0.000000
25%,27.000000,3.000000,5.000000,1.000000,33.000000,0.000000,2.000000,5.000000,1.000000,3.000000,-0.961200,-0.453700,2.000000,2.000000,6.000000,3.000000,0.070400
50%,37.000000,3.000000,10.000000,2.000000,47.000000,1.000000,3.000000,6.000000,2.000000,3.000000,-0.104800,-0.453700,2.000000,3.000000,7.000000,3.000000,0.142650
75%,52.000000,4.000000,21.000000,3.000000,61.000000,1.000000,3.000000,8.000000,3.000000,3.000000,0.751600,0.707500,2.000000,3.000000,8.000000,4.000000,0.235000
max,65.000000,5.000000,43.000000,4.000000,74.000000,2.000000,5.000000,10.000000,5.000000,5.000000,1.608100,1.868800,3.000000,5.000000,9.000000,5.000000,0.995900


In [ ]:
### Read the test set (for prediction and submission) ###
test_data = pd.read_csv("Test.csv")

In [ ]:
test_data.shape

(3000, 23)

In [ ]:
test_data.head()

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7
0,EID_22713,F,32.0,5,Single,Springfield,R&D,Conceptual,7.0,4,30,1,5,4.0,type2,1.0,3,-0.9612,-0.4537,2.0,1,8,4
1,EID_9658,M,65.0,2,Single,Lebanon,IT,Directive,41.0,2,72,1,1,1.0,type2,1.0,4,-0.9612,0.7075,1.0,2,8,2
2,EID_22203,M,52.0,3,Married,Springfield,Sales,Directive,21.0,3,25,0,1,8.0,type3,1.0,4,-0.1048,0.7075,2.0,1,9,3
3,EID_7652,M,50.0,5,Single,Washington,Marketing,Analytical,11.0,4,28,1,1,2.0,type0,4.0,3,-0.1048,0.7075,2.0,2,8,3
4,EID_6516,F,44.0,3,Married,Franklin,R&D,Conceptual,12.0,4,47,1,3,2.0,type2,4.0,4,1.6081,0.7075,2.0,2,7,4


In [ ]:
test_data.isnull().values.any()

True

In [ ]:
### Column/Feature wise missing vaues in the prediction set ###
test_data.isnull().sum()

Employee_ID                    0
Gender                         0
Age                          161
Education_Level                0
Relationship_Status            0
Hometown                       0
Unit                           0
Decision_skill_possess         0
Time_of_service               52
Time_since_promotion           0
growth_rate                    0
Travel_Rate                    0
Post_Level                     0
Pay_Scale                      3
Compensation_and_Benefits      0
Work_Life_balance              5
VAR1                           0
VAR2                         217
VAR3                           0
VAR4                         298
VAR5                           0
VAR6                           0
VAR7                           0
dtype: int64

In [ ]:
### Lets combine these two dataframes for summary stats and imputing the missing values ###
combined_data = pd.concat([train_data,test_data])

In [ ]:
combined_data.shape

(10000, 24)

In [ ]:
combined_data.tail()

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
2995,EID_22547,F,32.0,3,Single,Franklin,Sales,Directive,3.0,3,38,1,3,6.0,type2,2.0,1,-0.1048,0.7075,1.0,4,7,4,NaN
2996,EID_10066,F,NaN,2,Single,Franklin,Marketing,Conceptual,10.0,2,57,1,4,9.0,type2,1.0,3,0.7516,0.7075,3.0,4,8,3,NaN
2997,EID_7126,F,60.0,4,Single,Lebanon,Logistics,Analytical,35.0,1,69,0,2,9.0,type3,3.0,3,-1.8176,-0.4537,NaN,4,8,4,NaN
2998,EID_4929,F,51.0,1,Married,Springfield,IT,Behavioral,23.0,1,30,0,2,6.0,type3,1.0,5,0.7516,-0.4537,2.0,3,9,2,NaN
2999,EID_24750,F,45.0,1,Single,Franklin,Sales,Directive,12.0,4,70,1,2,8.0,type4,4.0,3,0.7516,-0.4537,2.0,2,8,5,NaN


In [ ]:
combined_data.isnull().sum()

Employee_ID                     0
Gender                          0
Age                           573
Education_Level                 0
Relationship_Status             0
Hometown                        0
Unit                            0
Decision_skill_possess          0
Time_of_service               196
Time_since_promotion            0
growth_rate                     0
Travel_Rate                     0
Post_Level                      0
Pay_Scale                      12
Compensation_and_Benefits       0
Work_Life_balance              16
VAR1                            0
VAR2                          794
VAR3                            0
VAR4                          954
VAR5                            0
VAR6                            0
VAR7                            0
Attrition_rate               3000
dtype: int64

In [ ]:
!pip install sweetviz

     |████████████████████████████████| 327kB 2.6MB/s 
     |████████████████████████████████| 71kB 3.6MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [ ]:
### Sweetviz will create a HTML report similar to pandas profiling feature ###
import sweetviz as sv

feature_config = sv.FeatureConfig(skip="Employee_ID")   # Removing Employee_ID as it is primary key here.

my_report = sv.compare([train_data,"Train"],[test_data,"Test"] ,'Attrition_rate', feature_config)
my_report.show_html("Report.html")

:FEATURES DONE:                    |█████████████████████| [100%]   00:10  -> (00:00 left)
:PAIRWISE DONE:                    |█████████████████████| [100%]   00:02  -> (00:00 left)


Creating Associations graph... DONE!


In [ ]:
### Imputing the missing values in Age column ###
median_age_on_g_rs = combined_data.groupby(['Gender', 'Relationship_Status'])['Age'].median().to_frame()

median_age_on_g_rs

Age
Gender Relationship_Status      
F      Married              38.0
       Single               38.0
M      Married              37.0
       Single               38.0

In [ ]:
### Lets create a function to fill the missing Age based on Relationship status and Gender ###
def fill_missing_age (cols):
    Age = cols[0]
    Gender = cols[1]
    Relationship_Status= cols[2]

    if (pd.isnull(Age)):
        if (Gender == 'F'):
            return 38
        elif (Gender == 'M'):
            if (Relationship_Status == 'Married'):
                return 37
            else: return 38
    else: return Age

In [ ]:
combined_data['Age']= combined_data[['Age','Gender','Relationship_Status']].apply(fill_missing_age, axis=1)

In [ ]:
### Lets check if Age column has any missing values. 0 means no missing values ###
combined_data['Age'].isnull().sum()

0

In [ ]:
### Time_of_service imputation ###
median_ts = combined_data['Time_of_service'].median()

median_ts

10.0

In [ ]:
combined_data['Time_of_service'] = combined_data['Time_of_service'].fillna(median_ts)

In [ ]:
combined_data['Time_of_service'].isnull().sum()

0

In [ ]:
### Pay scale imputation ###
mode_ps = math.floor(train_data['Pay_Scale'].mode())

mode_ps

8

In [ ]:
combined_data['Pay_Scale'] = combined_data['Pay_Scale'].fillna(mode_ps)

combined_data['Pay_Scale'].isnull().sum()

0

In [ ]:
### Work life balance imputation ###
mode_wlb = math.floor(combined_data['Work_Life_balance'].mode())

mode_wlb

1

In [ ]:
combined_data['Work_Life_balance'] = combined_data['Work_Life_balance'].fillna(mode_wlb)

combined_data['Work_Life_balance'].isnull().sum()

0

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_enc = LabelEncoder()

In [ ]:
### Label encoding the VAR2, VAR3 ###
combined_data['VAR2'] =lb_enc.fit_transform(combined_data['VAR2'])
combined_data['VAR3'] =lb_enc.fit_transform(combined_data['VAR3'])

In [ ]:
### VAR2 imputation. Var2 is a categoricla value column as observed in the Report.html ###
mode_var2 = math.floor(combined_data["VAR2"].mode())

mode_var2

3

In [ ]:
combined_data["VAR2"] = combined_data["VAR2"].fillna(mode_var2)

combined_data["VAR2"].isnull().sum()

0

In [ ]:
### VAR4 imputation ###
mode_var4 = math.floor(combined_data["VAR4"].mode())

mode_var4

2

In [ ]:
combined_data["VAR4"] = combined_data["VAR4"].fillna(mode_var2)

combined_data["VAR4"].isnull().sum()

0

In [ ]:
### Final check for any missing values. Ignoring the class column Attrition rate ###
combined_data.isnull().sum()

Employee_ID                     0
Gender                          0
Age                             0
Education_Level                 0
Relationship_Status             0
Hometown                        0
Unit                            0
Decision_skill_possess          0
Time_of_service                 0
Time_since_promotion            0
growth_rate                     0
Travel_Rate                     0
Post_Level                      0
Pay_Scale                       0
Compensation_and_Benefits       0
Work_Life_balance               0
VAR1                            0
VAR2                            0
VAR3                            0
VAR4                            0
VAR5                            0
VAR6                            0
VAR7                            0
Attrition_rate               3000
dtype: int64

In [ ]:
### Separate the training and test data ###
train_data = combined_data.iloc[0:7000,:]

In [ ]:
train_data.shape

(7000, 24)

In [ ]:
train_data.isnull().sum()

Employee_ID                  0
Gender                       0
Age                          0
Education_Level              0
Relationship_Status          0
Hometown                     0
Unit                         0
Decision_skill_possess       0
Time_of_service              0
Time_since_promotion         0
growth_rate                  0
Travel_Rate                  0
Post_Level                   0
Pay_Scale                    0
Compensation_and_Benefits    0
Work_Life_balance            0
VAR1                         0
VAR2                         0
VAR3                         0
VAR4                         0
VAR5                         0
VAR6                         0
VAR7                         0
Attrition_rate               0
dtype: int64

In [ ]:
test_data = combined_data.iloc[7000:,:]

In [ ]:
test_data.shape

(3000, 24)

In [ ]:
test_data.head()

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
0,EID_22713,F,32.0,5,Single,Springfield,R&D,Conceptual,7.0,4,30,1,5,4.0,type2,1.0,3,1,2,2.0,1,8,4,NaN
1,EID_9658,M,65.0,2,Single,Lebanon,IT,Directive,41.0,2,72,1,1,1.0,type2,1.0,4,1,3,1.0,2,8,2,NaN
2,EID_22203,M,52.0,3,Married,Springfield,Sales,Directive,21.0,3,25,0,1,8.0,type3,1.0,4,2,3,2.0,1,9,3,NaN
3,EID_7652,M,50.0,5,Single,Washington,Marketing,Analytical,11.0,4,28,1,1,2.0,type0,4.0,3,2,3,2.0,2,8,3,NaN
4,EID_6516,F,44.0,3,Married,Franklin,R&D,Conceptual,12.0,4,47,1,3,2.0,type2,4.0,4,4,3,2.0,2,7,4,NaN


In [ ]:
test_data.isnull().sum()

Employee_ID                     0
Gender                          0
Age                             0
Education_Level                 0
Relationship_Status             0
Hometown                        0
Unit                            0
Decision_skill_possess          0
Time_of_service                 0
Time_since_promotion            0
growth_rate                     0
Travel_Rate                     0
Post_Level                      0
Pay_Scale                       0
Compensation_and_Benefits       0
Work_Life_balance               0
VAR1                            0
VAR2                            0
VAR3                            0
VAR4                            0
VAR5                            0
VAR6                            0
VAR7                            0
Attrition_rate               3000
dtype: int64

In [ ]:
### Label encoding the columns for feature selection ###

temp_data = train_data.iloc[:,1:23]

In [ ]:
temp_data["Gender"] = lb_enc.fit_transform(temp_data['Gender'])
temp_data["Relationship_Status"] = lb_enc.fit_transform(temp_data['Relationship_Status'])
temp_data["Hometown"] = lb_enc.fit_transform(temp_data['Hometown'])
temp_data["Unit"] = lb_enc.fit_transform(temp_data['Unit'])
temp_data["Decision_skill_possess"] = lb_enc.fit_transform(temp_data['Decision_skill_possess'])
temp_data["Compensation_and_Benefits"] = lb_enc.fit_transform(temp_data['Compensation_and_Benefits'])

In [ ]:
temp_data

,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7
0,0,42.0,4,0,1,2,2,4.0,4,33,1,1,7.0,2,3.0,4,3,4,2.0,4,5,3
1,1,24.0,3,1,3,3,0,5.0,4,36,0,3,6.0,2,4.0,3,1,2,2.0,3,5,3
2,0,58.0,3,0,0,8,2,27.0,3,51,0,2,8.0,2,1.0,4,1,2,3.0,3,8,3
3,0,26.0,3,1,2,1,1,4.0,3,56,1,3,8.0,2,1.0,3,0,2,3.0,3,7,3
4,0,31.0,1,0,3,3,2,5.0,4,62,1,3,2.0,3,3.0,1,3,2,2.0,2,8,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,0,23.0,5,0,1,5,1,4.0,1,20,1,2,6.0,3,3.0,1,3,2,2.0,3,7,3
6996,0,44.0,1,0,2,9,0,16.0,2,59,0,2,4.0,3,3.0,2,2,0,2.0,4,8,4
6997,0,49.0,3,1,3,2,3,10.0,1,59,1,2,3.0,2,4.0,2,2,2,2.0,4,8,3
6998,0,47.0,3,0,4,10,1,24.0,1,21,1,5,8.0,0,2.0,4,3,4,2.0,3,6,3


In [ ]:
### Feature selection ###

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [ ]:
X = temp_data

In [ ]:
y = train_data["Attrition_rate"]

In [ ]:
############# f_regression ##########

bestfeatures = SelectKBest(score_func= f_regression, k=10)
fit = bestfeatures.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns

featureScores

,Specs,Score
0,Gender,5.706395
1,Age,1.576945
2,Education_Level,0.464089
3,Relationship_Status,0.580407
4,Hometown,0.322075
5,Unit,4.143021
6,Decision_skill_possess,2.294394
7,Time_of_service,2.071546
8,Time_since_promotion,1.348465
9,growth_rate,1.420629


In [ ]:
from sklearn.feature_selection import mutual_info_regression

In [ ]:
bestfeatures1 = SelectKBest(score_func= mutual_info_regression, k=10)
fit1 = bestfeatures1.fit(X,y)

dfscores1 = pd.DataFrame(fit1.scores_)
dfcolumns1 = pd.DataFrame(X.columns)

featureScores1 = pd.concat([dfcolumns1,dfscores1],axis=1)
featureScores1.columns = ['Specs','Score']  #naming the dataframe columns

featureScores1

In [ ]:
from yellowbrick.target import FeatureCorrelation


# Create a list of the feature names
features = np.array(temp_data.columns)

# Instantiate the visualizer
visualizer = FeatureCorrelation(labels=features)

visualizer.fit(X, y)        # Fit the data to the visualizer
visualizer.show()

In [ ]:
discrete = [False for _ in range(len(features))]
discrete[1] = True
discrete

In [ ]:
# Instantiate the visualizer
visualizer = FeatureCorrelation(method='mutual_info-regression', labels=features)

visualizer.fit(X, y, discrete_features= discrete, random_state=0)
visualizer.show()

In [ ]:
### Create dataframe with only selected top K features ###
X_data = X[['Gender','Unit','Decision_skill_possess','Time_of_service','Compensation_and_Benefits',
            'Work_Life_balance','VAR4']]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data, y, test_size=0.25, random_state= 42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_rfr = RandomForestRegressor(n_estimators= 100,random_state = 42 ,max_depth= 3 , min_samples_leaf= 5)

model_rfr.fit(X_train, Y_train)

y_rfr_pred = model_rfr.predict(X_test)

In [ ]:
mse_rfr = sqrt(mean_squared_error(Y_test, y_rfr_pred))
print("\n Root mean squared error (based on f_regressor values model): ", mse_rfr)

acc_score = 100* max(0, 1- mse_rfr)

print("\n Result for all feature model (based on f_regressor values model): ", acc_score)


 Root mean squared error (based on f_regressor values model):  0.1882645475349247

 Result for all feature model (based on f_regressor values model):  81.17354524650753


In [ ]:
### grid search for the optimisation ###
from sklearn.model_selection import GridSearchCV

parameters_rf = {
    'n_estimators':[20,25,50,100,150],
    'random_state':[42],
    'max_depth':[3,5,10,15,20,30],
    'min_samples_leaf':[3,5,10],
    'max_samples': [0.3,0.5,0.75,0.9,1.0],
    'max_features':['log2','sqrt','auto']
}




gsearch_rf = GridSearchCV(estimator = model_rfr , param_grid = parameters_rf,
                          n_jobs=-1,iid=False, verbose=10,scoring='neg_mean_squared_error')

gsearch_rf.fit(X_data, y)

Fitting 5 folds for each of 1350 candidates, totalling 6750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1929s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 184 tas

GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=3,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=5,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, rand...
                                             verbose=0, warm_start=False),
             iid=False, n_jobs=-1,
           

In [ ]:
print('best params')
print (gsearch_rf.best_params_)

print('best score')
print (gsearch_rf.best_score_)

best params
{'max_depth': 3, 'max_features': 'log2', 'max_samples': 0.75, 'min_samples_leaf': 5, 'n_estimators': 25, 'random_state': 42}
best score
-0.034439338741733536


In [ ]:
### Model with optimised parameters ###
new_model_rfr = RandomForestRegressor(n_estimators= 25,random_state = 42 ,max_depth= 3 , min_samples_leaf= 5,
                                 max_samples = 0.75,max_features= 'log2' )

new_model_rfr.fit(X_train, Y_train)

new_y_rfr_pred = new_model_rfr.predict(X_test)

In [ ]:
new_mse_rfr = sqrt(mean_squared_error(Y_test, new_y_rfr_pred))
print("\n Root mean squared error (based on f_regressor values model): ", new_mse_rfr)

new_acc_score = 100* max(0, 1- new_mse_rfr)

print("\n Result for all feature model (based on f_regressor values model): ", new_acc_score)


 Root mean squared error (based on f_regressor values model):  0.18823955055276204

 Result for all feature model (based on f_regressor values model):  81.17604494472378
